#  Shadow Model

The adversary trains $k$ shadow models $f^i_{shadow}()$, each on a dataset (shadow data) that is similar
in format and distribution as the __target model's private training set__.

Generate shadow data drawing samples from the population where the target model's training data
are drawn from. The shadow model must be __trained in a similar fashion as the target model__.
The larger the number of shadow models, the better the accuracy of the attack model.

In [1]:
%run ../shadow_model.py

In [2]:
import numpy as np

In [9]:
arr = np.arange(20).reshape(4,5)

In [10]:
np.random.shuffle(arr)

In [11]:
arr

array([[10, 11, 12, 13, 14],
       [ 0,  1,  2,  3,  4],
       [15, 16, 17, 18, 19],
       [ 5,  6,  7,  8,  9]])